In [4]:
import numpy as np
import pandas as pd
# Load data from CSV
file_path = 'Advertising.csv'
df = pd. read_csv ( file_path )
print (df. head ())

   sample index     TV  radio  newspaper  sales
0             1  230.1   37.8       69.2   22.1
1             2   44.5   39.3       45.1   10.4
2             3   17.2   45.9       69.3    9.3
3             4  151.5   41.3       58.5   18.5
4             5  180.8   10.8       58.4   12.9


In [5]:
from sklearn.model_selection import train_test_split


X = df.drop("sales", axis=1)  # Features (TV, radio, newspaper)
y = df["sales"]  # Target variable (sales)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets to verify the split
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (160, 4)
X_test shape: (40, 4)
y_train shape: (160,)
y_test shape: (40,)


In [7]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Print the coefficients
coefficients = model.coef_
intercept = model.intercept_

print("Intercept:", intercept)
print("Coefficients:")
for feature, coef in zip(X.columns, coefficients):
    print(f"{feature}: {coef:.2f}")


Intercept: 2.9065270863618125
Coefficients:
sample index: 0.00
TV: 0.04
radio: 0.19
newspaper: 0.00


In [10]:

from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

# Predict on training and testing data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate Residual Sum of Squares (RSS)
RSS_train = np.sum((y_train - y_train_pred)**2)
RSS_test = np.sum((y_test - y_test_pred)**2)

# Calculate Residual Standard Error (RSE)
N_train, d_train = X_train.shape[0], X_train.shape[1] + 1  # +1 for intercept
N_test, d_test = X_test.shape[0], X_test.shape[1] + 1  # +1 for intercept
RSE_train = np.sqrt(RSS_train / (N_train - d_train))
RSE_test = np.sqrt(RSS_test / (N_test - d_test))

# Calculate Mean Squared Error (MSE)
MSE_train = mean_squared_error(y_train, y_train_pred)
MSE_test = mean_squared_error(y_test, y_test_pred)

# Calculate R^2 statistic
R2_train = r2_score(y_train, y_train_pred)
R2_test = r2_score(y_test, y_test_pred)

# Calculate the standard errors, t-statistics, and p-values for each feature
X_train_with_intercept = np.column_stack([np.ones(N_train), X_train])
X_test_with_intercept = np.column_stack([np.ones(N_test), X_test])

# Compute the residuals
residuals_train = y_train - y_train_pred
residuals_test = y_test - y_test_pred

# Calculate standard error for each feature
XtX_inv_train = np.linalg.inv(np.dot(X_train_with_intercept.T, X_train_with_intercept))
SE_coefs_train = np.sqrt(np.diagonal(RSE_train**2 * XtX_inv_train))

# Calculate t-statistic and p-value for each feature
t_stats_train = model.coef_ / SE_coefs_train[1:]  # Exclude the intercept
p_values_train = [2 * (1 - stats.t.cdf(np.abs(t), N_train - d_train - 1)) for t in t_stats_train]

# Print the calculated statistics
print("Training Data:")
print("RSS:", RSS_train)
print("RSE:", RSE_train)
print("MSE:", MSE_train)
print("R^2:", R2_train)
print("Standard Errors for Coefficients:")
for feature, SE in zip(X.columns, SE_coefs_train[1:]):  # Exclude the intercept
    print(f"{feature}: {SE:.2f}")
print("t-statistics for Coefficients:")
for feature, t_stat in zip(X.columns, t_stats_train):
    print(f"{feature}: {t_stat:.2f}")
print("p-values for Coefficients:")
for feature, p_value in zip(X.columns, p_values_train):
    print(f"{feature}: {p_value:.6f}")

print("\nTesting Data:")
print("RSS:", RSS_test)
print("RSE:", RSE_test)
print("MSE:", MSE_test)
print("R^2:", R2_test)


Training Data:
RSS: 432.5946558356594
RSE: 1.6706086505988968
MSE: 2.7037165989728713
R^2: 0.8957553000540606
Standard Errors for Coefficients:
sample index: 0.00
TV: 0.00
radio: 0.01
newspaper: 0.01
t-statistics for Coefficients:
sample index: 0.28
TV: 28.44
radio: 19.46
newspaper: 0.43
p-values for Coefficients:
sample index: 0.776336
TV: 0.000000
radio: 0.000000
newspaper: 0.670249

Testing Data:
RSS: 127.96017874355627
RSE: 1.91206827988928
MSE: 3.1990044685889067
R^2: 0.898648915141708


In [14]:

coefficient_tv = 0.04
coefficient_radio = 0.19

# Budget allocation for scenarios
budget_tv_scenario1 = 25000  # $25,000 for TV advertising in scenario 1
budget_radio_scenario1 = 25000  # $25,000 for radio advertising in scenario 1
budget_tv_scenario2 = 50000  # $50,000 for TV advertising in scenario 2
budget_radio_scenario2 = 0  # No budget for radio advertising in scenario 2

# Calculate predicted sales for scenario 1
predicted_sales_scenario1 = coefficient_tv * budget_tv_scenario1 + coefficient_radio * budget_radio_scenario1

# Calculate predicted sales for scenario 2 (TV only)
predicted_sales_scenario2_tv = coefficient_tv * budget_tv_scenario2

# Calculate predicted sales for scenario 2 (Radio only)
predicted_sales_scenario2_radio = coefficient_radio * budget_radio_scenario2

# Print the results
print("Scenario 1: Allocating $25,000 to both TV and Radio")
print(f"Predicted sales: ${predicted_sales_scenario1:.2f}\n")

print("Scenario 2: Allocating $50,000 to TV or $50,000 to Radio")
print(f"Predicted sales (TV): ${predicted_sales_scenario2_tv:.2f}")
print(f"Predicted sales (Radio): ${predicted_sales_scenario2_radio:.2f}")


Scenario 1: Allocating $25,000 to both TV and Radio
Predicted sales: $5750.00

Scenario 2: Allocating $50,000 to TV or $50,000 to Radio
Predicted sales (TV): $2000.00
Predicted sales (Radio): $0.00
